# Проект "Аналитика благотворительного фонда АиФ Доброе сердце"

**Описание:**

Благотворительный фонд "АиФ Доброе сердце" хочет лучше узнать своих благотворителей, чтобы максимально эффективно выстроить свою работу с ними. 
У фонда есть «старожилы» - доноры, которые достаточно давно с фондом и одной из задач будет изучение их поведения: активны ли они, делают ли пожертвования и как реагируют на активность фонда?  

**Цели исследования:**

- проанализировать имеющиеся данные; 
- проанализировать поведение доноров: в целом, исторических пользователей и новых пользователей отдельно; 
- составить портрет типичного пользователя: регион, поведение, др. параметры; 
- рассчитать метрики: пользовательские, маркетинговые, коммерческие;
- сегментировать пользователей: RFM-анализ, описание сегментов;
- изучить каналы привлечения пользователей и их эффективность;
- изучить эффективность рассылок.

**Задачи:**

- провести исследовательский анализ данных;
- составить RFM - анализ пользователей;
- посчитать метрики пользователей;
- формулирование выводов по полученным результатам, подготовка презентации.




**Ход исследовия:**

- прочитать данные из файла и сохранить в датафремы;
- обзор и преобработка входных данных;
- исследовательский анализ данных;
- расчет метрик;
- RFM - анализ;
- когортный анализ;
- проверка гипотез;
- формулирование выводов по полученным результатам;
- подготовка презентацииации.

## Описание данных

**Файл** `id_donor.csv` - информация о донорах:

- `CustomerCustomFieldsRecurrent` - рекуррент да / нет;
- `CustomerCustomFieldsVolunteer` - волонтер да / нет;
- `CustomerSex` - пол пользователя;
- `CustomerAreaIdsExternalId` - идентификатор региона пользователя;
- `CustomerAreaName` - название региона пользователя;
- `CustomerIanaTimeZone` - часовой пояс пользователя;
- `CustomerTimeZoneSource` - источник данных о часовом поясе;
- `CustomerIdsMindboxId` - идентификатор пользователя;
- `CustomerIsEmailInvalid` - почта невалидна да / нет;
- `CustomerChangeDateTimeUtc` - дата регистрации / редактирования;
- `CustomerCustomerSubscriptions DobroaifIsSubscribed` - подписка;
- `CustomerCustomerSubscriptions DobroaifSmsIsSubscribed` - подписка в точке контакта SMS;
- `CustomerCustomerSubscriptions DobroaifEmailIsSubscribed` - подписка в точке контакта email;
- `CustomerCustomerSubscriptions DobroaifViberIsSubscribed` - подписка в точке контакта Viber;
- `CustomerCustomerSubscriptions DobroaifMobilePushIsSubscribed` - подписка в точке контакта Mobile Push;
- `CustomerCustomerSubscriptions DobroaifWebPushIsSubscribed` - подписка в точке контакта Web Push.

 
**Файл** `order.csv` - информация о пожертвованиях:

- `OrderIdsMindboxId` - идентификатор платежа;
- `OrderFirstActionIdsMindboxId` - идентификатор действия;
- `OrderFirstActionDateTimeUtc` - дата и время оформления заказа;
- `OrderFirstActionChannelIdsMindboxId` - идентификатор точки контакта;
- `OrderFirstActionChannelIdsExternalId` - внешний идентификатор точки контакта;
- `OrderFirstActionChannelName` - наименование точки контакта;
- `OrderAreaIdsExternalId` - идентификатор региона пользователя;
- `OrderTransactionIdsExternalId` - внешний идентификатор транзакции;
- `OrderTotalPrice` - стоимость заказа;
- `OrderIdsWebsiteID` - идентификатор заказа на сайте;
- `OrderCustomFieldsNewyear` - наличие поля с новым годом;
- `OrderCustomFieldsNextPayDate` - дата следующего списания денег;
- `OrderCustomFieldsRecurrent` - рекурент да / нет;
- `OrderCustomFieldsRepayment` - повторный платеж;
- `OrderLineProductIdsWebsite` - идентификатор продукта на сайте;
- `OrderLineProductName` - название продукта;
- `OrderLineQuantity` - количество единиц продукта;
- `OrderLineBasePricePerItem` - базовая цена за единицу продукта;
- `OrderLinePriceOfLine` - итоговая цена;
- `OrderLineStatusIdsExternalId` - идентификатор статуса позиции заказа;
- `OrderCustomerIdsMindboxId` - идентификатор пользователя.


**Файл** `import.csv` - информация об историческом импорте:

- `CustomerActionCustomerIdsMindboxId` - идентификатор пользователя.


**Файл** `channels.pkl` - информация о каналах привлечения:

- `User_action` - название действия пользователя;
- `Action_date` - дата действия пользователя;
- `Channel_id` - идентификатор канала (описание в текстовом файле);
- `Utm_campaign` - кампания;
- `Utm_source` - ресурс;
- `Utm_medium` - что-то дублирующееся;
- `User_id` - идентификатор пользователя;
- `Action_time` - час действия пользователя.

 


## Настройка рабочей среды

In [1]:
# Подгрузка модулей
import json
import math
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re
import scipy.stats as stats
import seaborn as sns

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from matplotlib import gridspec
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots

from operator import attrgetter

#from ydata_profiling import ProfileReport

import warnings
warnings.simplefilter("ignore", category=FutureWarning)

## Глобальные переменные

In [2]:
path_dir = 'datasets/'

start_date = datetime(2024, 3, 19)

# Уровень статистической значимости
alpha = 0.05

# момент анализа
observation_date = start_date.date()
# горизонт анализа
horizon_months = 14  


pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = None
pd.set_option('future.no_silent_downcasting', True)

dict_part_of_day = {
    0: 'ночь - 0:00-6:00', 
    1: 'утро - 6:00-12:00', 
    2: 'день - 12:00-18:00', 
    3: 'вечер - 18:00-24:00', 
}


In [3]:
# Данные для RFM - анализа
rfm_data = pd.DataFrame(
    {
        'RFM': [
            '111', '112', '113', '121', '122', '123', '131', '132', '133', '211', '212', '213',
            '221', '222', '223', '231', '232', '233', '311', '312', '313', '321', '322', '323',
            '331', '332', '333'
        ],
        'Определение сегмента': [
            'Недавно покупавшие частые с высоким чеком', 'Недавно покупавшие частые со средним чеком', 
            'Недавно покупавшие частые с низким чеком', 'Недавно покупавшие редкие с высоким чеком', 
            'Недавно покупавшие редкие со средним чеком', 'Недавно покупавшие редкие с низким чеком', 
            'Недавно покупавшие разовые с высоким чеком', 'Недавно покупавшие разовые со средним чеком', 
            'Недавно покупавшие разовые с низким чеком', 'Спящие частые с высоким чеком', 
            'Спящие частые со средним чеком', 'Спящие частые с низким чеком', 
            'Спящие редкие с высоким чеком', 'Спящие редкие со средним чеком', 
            'Спящие редкие с низким чеком', 'Спящие разовые с высоким чеком', 
            'Спящие разовые со средним чеком', 'Спящие разовые с низким чеком', 
            'Давние частые с высоким чеком', 'Давние частые со средним чеком', 
            'Давние частые с низким чеком', 'Давние редкие с высоким чеком', 
            'Давние редкие со средним чеком', 'Давние редкие с низким чеком', 
            'Давние разовые с высоким чеком', 'Давние разовые со средним чеком', 
            'Давние разовые с низким чеком'
        ],
        'Определение группы': [
            'VIP', 'Выгодные', 'Выгодные', 'Потенциально выгодные', 'Потенциально выгодные', 'Новенькие', 
            'Новенькие', 'Новенькие', 'Новенькие', 'Спящие выгодные', 'Спящие выгодные', 'Спящие выгодные', 
            'Спящие выгодные', 'Спящие выгодные', 'Спящие', 'Спящие', 'Спящие', 'Спящие', 'Уходящие выгодные', 
            'Уходящие выгодные', 'Уходящие', 'Уходящие', 'Уходящие', 'Потерянные', 'Потерянные', 
            'Потерянные', 'Потерянные'
        ]
    }
)

## Функции

### Функции предварительно обработки данных

#### Функция df_analysis()

Функция для предварительного общего анализа датафрейма.

In [4]:
# Функция для предварительного общего анализа датафрейма
def df_analysis(df):

    # Отображение датафрейма df
    display(df.head())
    print('*'*100)

    # Вывод основной информации о датафрейме df с помощью метода info().
    display(df.info())
    print('*'*100)

    # Проверка количества пропусков в датафрейме  df
    isna = df.isna().sum().reset_index(name='cnt').query('cnt > 0')
    if isna.shape[0] == 0:
        print('Пропусков в датафрейме нет.')  
    else:
        print('Пропуски в датафрейме:')  
        display(isna)
    print('*'*100)
    
    # Проверка количества дубликатов в датафрейме  df
    duplicated = df.duplicated().sum()
    print('Количество дубликатов в датафрейме: {}'.format(duplicated), end='')
    # Подсчет процента дубликатов в датафрейме df
    print(' ({:.2%} от общего количества записей).'.format(duplicated/df.shape[0]))    
    print('*'*100)


#### Функция df_column_astype()

Функция для преобразования типа данных у списка столбцов.

In [5]:
def df_column_astype(df, change_type):

    for column in change_type:
        df[column] = df[column].astype(change_type[column])


####  Функция df_describe_by_parameter()

Функция для получения статистических характеристик по столбцу

In [6]:
def df_describe_by_parameter(df, column):

    cnt = int((df[column].describe())['count'])
    q1 = round((df[column].describe())['25%'], 2)
    q3 = round((df[column].describe())['75%'], 2)
    mean = round((df[column].describe())['mean'], 2)
    median = round((df[column].describe())['50%'], 2)
    q_min = round((df[column].describe())['min'], 2)
    q_max = round((df[column].describe())['max'], 2)

    print('Всего значений: {0}'.format(cnt))
    print()
    print('Среднее арифметическое значение: {0}'.format(mean))
    print('Медианное значение: {0}'.format(median))
    print('В среднем значения варьируются от {0} до {1}'.format(q1, q3))
    print('Минимальное значение {0}'.format(q_min))   
    print('Максимальное значение {0}'.format(q_max))        

#### Функция df_duplicated_info()

Функция для поиска статистики по дубликатам в датафрейме.

In [7]:
def df_duplicated_info(df):
    # Проверка количества дубликатов в датафрейме  df
    cnt = df.duplicated().sum()

    print('Количество дубликатов в датафрейме: {0}'.format(cnt), end='')

    # Подсчет процента дубликатов в датафрейме df
    print(' ({:.2%} от общего количества записей).'.format(cnt/df.shape[0]))    

#### Функция df_isna_info()

Функция для поиска статистики по пропускам в датафрейме.

In [8]:
def df_isna_info(df):
    # Проверка количества пропусков в датафрейме  df
    isna = df.isna().sum().reset_index(name='cnt').query('cnt > 0')
    isna['percent'] = ((isna['cnt'] / len(df)) * 100).map(lambda x: f"{x:.2f}%")
    
    if isna.shape[0] == 0:
        print('Пропусков в датафрейме нет.')  
    else:
        print('Пропуски в датафрейме:')  
        display(isna)

#### Функция read_csv_file()

In [9]:
def read_csv_file(file_name):
    # Чтение только заголовков
    headers = pd.read_csv(file_name, encoding='cp1251', sep=';', nrows=0).columns
    
    # Обработка названий столбцов: удаление всего после знака " ="
    cleaned_headers = [h.split(' =')[0].strip() for h in headers]
    
    # Загрузка данных с обновлёнными названиями столбцов
    return pd.read_csv(file_name, encoding='cp1251', sep=';', names=cleaned_headers, header=0, low_memory=False)


#### Функция read_json_file()

In [10]:
def read_json_file(file_name):
    # Чтение содержимого файла
    with open(file_name, 'r', encoding='utf-8') as file:
        file_content = file.read().replace("'", '"')

    # Загрузка JSON и преобразование ключей в числа
    data = json.loads(file_content, object_pairs_hook=lambda x: {int(k): v for k, v in x})
    
    return data


#### Функция rank_value()



In [11]:
def rank_value(df, column):
    
    q1 = (df[column].describe())['25%']
    q3 = (df[column].describe())['75%']
    
    # Верхняя граница "усов" размаха
    iqr_mx = round(q3 + 1.5 * (q3 - q1), 2)

    bins = [0] + list(np.percentile(df.loc[df[column] < iqr_mx][column], [33, 66])) + [df[column].max()]
    print('Разбиение клиентов на 3 сегмента по значению "{}": '.format(column), end='')

    print(bins)
    
    return bins

#### Функция get_part_of_day()

In [12]:
# Функция для определения части суток
def get_part_of_day(hour):
    if 0 <= hour < 6:
        return 0
    elif 6 <= hour < 12:
        return 1
    elif 12 <= hour < 18:
        return 2
    else:
        return 3

#### Функция replace_sex()

In [13]:
def replace_sex(value):
    if value == 'female':
        return 'Ж'
    elif value == 'male':
        return 'М'
    elif pd.isna(value):
        return 'неизвестно'
    else:
        return value



#### Функция replace_bool()

In [14]:
def replace_bool(value):
    if value == False:
        return 'Нет'
    elif value == True:
        return 'Да'
    else:
        return value


#### Функция check_ttest_ind

In [15]:
def check_ttest_ind(df1, df2, column, alternative):

    results = stats.ttest_ind(df1[column], df2[column], alternative=alternative, equal_var=False)
    
    print(f'p-value: {results.pvalue}')
    
    # проверяем p-value
    if results.pvalue < alpha:
        print('Отвергаем нулевую гипотезу')
    else:
        print('Нет оснований отвергнуть нулевую гипотезу')

In [16]:
def transliterate(name):
    cyrillic_to_latin = {
        'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'д': 'd', 'е': 'e', 'ё': 'e', 'ж': 'zh',
        'з': 'z', 'и': 'i', 'й': 'j', 'к': 'k', 'л': 'l', 'м': 'm', 'н': 'n', 'о': 'o',
        'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'у': 'u', 'ф': 'f', 'х': 'kh', 'ц': 'ts',
        'ч': 'ch', 'ш': 'sh', 'щ': 'shch', 'ы': 'y', 'э': 'e', 'ю': 'yu', 'я': 'ya', 'ь' : ''
    }
    transliterated_name = ''.join(cyrillic_to_latin.get(char, char) for char in name)
    return transliterated_name

def put_campaing_name(input_string):
    # Шаблон для удаления конструкции
    pattern = r'Пожертвование\s+Благотворительный\s+фонд\s+«АиФ\.\s+Доброе сердце»\s+'
    
    # Замена шаблонной части строки на пустую строку
    cleaned_string = re.sub(pattern, '', input_string)
    
    # Проверка, выглядит ли строка как "Имя Фамилия"
    if re.match(r'^[А-ЯЁ][а-яё]+\s[А-ЯЁ][а-яё]+$', cleaned_string):
        # Преобразование строки в транслит
        transliterated_string = transliterate(cleaned_string.lower())
        # Замена пробелов на дефисы
        transliterated_string = transliterated_string.replace(' ', '-')
        return transliterated_string
    
    return cleaned_string



In [17]:
# Функция для проверки, является ли строка транслитом "имя-фамилия"
def is_translit_name(name):
    pattern = r'^[a-z]+-[a-z]+$'
    return re.match(pattern, name) is not None


### Функции для графического отображения информации

#### Функция draw_bar()

Функция для построения столбчатой диаграммы

In [18]:
# Функция для построения столбчатой диаграммы
def draw_bar(df, x, y, title, xlabel, ylabel, is_top=False):

    # Рассчитываем долю каждого значения
    df['percentage'] = df[y] / df[y].sum() * 100
 
    # Если нужно отобрать только топ-10 записей
    if is_top:
        df = df.sort_values(by=y, ascending=False).head(10)
    else:
        df = df.sort_values(by=y, ascending=False)
    
    # Простроение графика
    fig = go.Figure()
    
    # Добавление столбчатой диаграммы
    fig.add_trace(
        go.Bar(
            x=df[x], 
            y=df[y], 
            opacity=0.5, 
            text=df[y], 
            hovertemplate='%{x}<br>Значение: %{text}<br>Доля: %{customdata:.2f}%<extra></extra>', 
            customdata=df['percentage'], 
            showlegend=False,
            hoverinfo="skip"
        )
    )
    
    # Оформление графика
    fig.update_layout(
            xaxis_title=xlabel, 
            yaxis_title=ylabel, 
            title=title
    )

    # Вывод графика
    fig.show() 


#### Функция draw_bar_compare()

Функция для построения двух столбчатых диаграмм для сравнения

In [19]:
# Функция для построения двух столбчатых диаграмм для сравнения
def draw_bar_compare(df, x, y1, y2, title, subtitle1, subtitle2, x_lable, y_lable):
    
    # Создаем график
    fig = go.Figure()

    # Построение первой столбчатой диаграммы
    fig.add_trace(px.bar(df, x=x, y=y1, opacity=0.5, color_discrete_sequence=['blue'], labels={y1: subtitle1}, barmode='overlay').data[0])

    # Построение второй столбчатой диаграммы
    fig.add_trace(px.bar(df, x=x, y=y2, opacity=0.5, color_discrete_sequence=['green'], labels={y2: subtitle2}, barmode='overlay').data[0])

    # Оформление графика
    fig.update_layout(title=title, xaxis_title=x_lable, yaxis_title=y_lable)

    # Вывод графика
    fig.show() 


#### Функция draw_bar_horizontal()

Функция для построения горизонтальной столбчатой диаграммы

In [20]:
# Функция для построения горизонтальной столбчатой диаграммы
def draw_bar_horizontal(df, x, y, title, xlabel, ylabel, is_top=False):

    # Рассчитываем долю каждого значения
    df['percentage'] = df[y] / df[y].sum() * 100
 
    # Если нужно отобрать только топ-10 записей
    if is_top:
        df = df.sort_values(by=y, ascending=True).tail(10)
    else:
        df = df.sort_values(by=y, ascending=True)
    
    # Простроение графика
    fig = go.Figure()
    
    # Добавление горизонтальной столбчатой диаграммы
    fig.add_trace(
        go.Bar(
            x=df[y], 
            y=df[x], 
            orientation='h', 
            opacity=0.5,
            text=df[y], 
            hovertemplate='%{y}<br>Значение: %{text}<br>Доля: %{customdata:.2f}%<extra></extra>', 
            customdata=df['percentage'], 
            showlegend=False,
            hoverinfo="skip"
        )
    )
    
    # Оформление графика
    fig.update_layout(
        xaxis_title=ylabel, 
        yaxis_title=xlabel, 
        title=title, 
    )

    # Вывод графика
    fig.show()



#### Функция draw_box()

Функция для построения диаграммы размаха.

In [21]:
# функция для построения графика boxplot
def draw_box(df, x, title, x_lim=0):

    # Простроение графика
    fig = go.Figure()

    fig = px.box(df, x=x)
        
    if x_lim:
        title +=' (limit X)'
        fig.update_xaxes(range=[0, x_lim])

    # Оформление графика
    fig.update_layout(title=title, xaxis_title='Значение параметра', showlegend=True)
    
    # Вывод графика
    fig.show()
    
    df_describe_by_parameter(df, x)
    

#### Функция draw_heatmap()

Функция для построения heatmap.

In [22]:
# Функция для построения heatmap
def draw_heatmap(df, title, xlabel, ylable, annot=True, size_x=12, size_y=7):
    
    # Размер графика
    plt.figure(figsize = (size_x, size_y))
    
    # Простроение графика
    sns.heatmap(df, annot = annot, fmt = '.2%')

    # Оформление графика
    plt.title(title)  
    plt.xlabel(xlabel) 
    plt.ylabel(ylable) 

    # Вывод графика
    plt.show()  


#### Функция draw_pie()

Функция для построения круговой диаграммы

In [23]:
# Функция для построения круговой диаграммы
def draw_pie(df, x, y, title):

    df[x] = df[x].map(replace_bool)
    
    # Простроение графика
    fig = go.Figure(
        go.Pie(
                labels=df[x], 
                values=df[y], 
        )
    )

    # Оформление графика
    fig.update_layout(title=title, showlegend=True )

    # Вывод графика
    fig.show()


#### Функция draw_pie_compare()

Функция для построения двух круговых диаграмм для сравнения

In [24]:
# Функция для построения двух круговых диаграмм рядом для сравнение
def draw_pie_compare(df1, df2, x, y, title, subtitle1, subtitle2):
    
    df1[x] = df1[x].map(replace_bool)
    
    df2[x] = df2[x].map(replace_bool)
    
    # Построение круговых диаграмм
    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
    
    fig.add_trace(go.Pie(labels=df1[x], values=df1[y],name=subtitle1), 1, 1)
    fig.add_annotation(text=subtitle1, x=0.2, y=-0.1, showarrow=False)
    
    fig.add_trace(go.Pie(labels=df2[x], values=df2[y], name=subtitle2), 1,2)
    fig.add_annotation(text=subtitle2, x=0.8, y=-0.1, showarrow=False)
    
    # Оформление графика
    fig.update_layout(title_text=title, title_x=0.5)

    # Вывод графика
    fig.show()

#### Функция draw_plot()

Функция построения plot графика

In [25]:
def draw_plot(df, title, xlabel, ylabel):
    
    # Простроение графика
    fig = go.Figure()
    
    # Цветовая палитра
    palette = px.colors.qualitative.Plotly
    
    # Добавление данных на график
    for i, field in enumerate(df.columns):
            fig.add_trace(
                go.Scatter(
                    x=df.index,
                    y=df[field],
                    mode='lines',
                    name=field,
                    line=dict(color=palette[i % len(palette)])
                )
            )
    
    # Оформление графика
    fig.update_layout(
        title=title,
        xaxis_title=xlabel,
        yaxis_title=ylabel,
        legend_title="Легенда"
    )
    
    # Вывод графика
    fig.show()



#### Функция draw_funnel()

Функция для построения воронки

In [26]:
# Функция для построения воронки
def draw_funnel(df, x, y, title):
    
    fig = go.Figure(
        go.Funnel(
                x = df[x],
                y = df[y],
                textinfo = 'value + percent previous',
                marker = {'color': ['blue', 'green', 'lightblue']}
        )
    )

    # Оформление графика
    fig.update_layout(
                title_text=title, 
                title_x=0.5
    )
    
    # Вывод графика
    fig.show()     


#### Функция draw_T_plot()

Функция для построения plot-графиков по траспонированным колонкам

In [27]:
def draw_T_plot(data, title, xlabel, ylabel):
    # Создаем фигуру
    fig = go.Figure()
    
    # Преобразование индекса в строку, если это PeriodIndex
    if isinstance(data.index, pd.PeriodIndex):
        data.index = data.index.astype(str)
        
    # Добавляем линии для каждой строки (транспонированных колонок)
    for row in data.index:
        fig.add_trace(
            go.Scatter(
                x=data.columns,
                y=data.loc[row],
                mode='lines',
                name=row
            )
        )
    
      
    # Оформление графика
    fig.update_layout(
        title=title,
        xaxis_title=xlabel,
        yaxis_title=ylabel,
        xaxis=dict(tickmode='array', tickvals=data.index, ticktext=data.index),
        legend=dict(x=1, y=1)  # Легенда за пределами графика
    )
    
    # Вывод графика
    fig.show()


## Загрузка данных и изучение общей информации

In [ ]:
# Чтение данных из csv-файлов в датафрейм и сохранение в переменные
try:
    donors = read_csv_file(path_dir + 'id_donor.csv')
    orders = read_csv_file(path_dir + 'order.csv')
    imports = read_csv_file(path_dir + 'import.csv')
    channels = pd.read_pickle(path_dir + 'channels.pkl')
    dic_channels = read_json_file(path_dir + 'channels_dict.txt')
except FileNotFoundError as e:
    print(f"FileNotFoundError successfully handled\n"
          f"{e}")

## Предварительный анализ данных

### is_donor.csv

In [ ]:
#ProfileReport(donors, title="Pandas Profiling id_donor.csv", explorative=True)


### orders.csv

In [ ]:
#ProfileReport(orders, title="Pandas Profiling order.csv", explorative=True)


### imports.csv

In [ ]:
#ProfileReport(imports, title="Pandas Profiling import.csv", explorative=True)


### channels.pkl

In [ ]:
channels['action_date'] = pd.to_datetime(channels['action_date'])

#ProfileReport(channels, title="Pandas Profiling channels.pkl", explorative=True)


## Предобработка данных


### Переименование столбцов

In [ ]:
donors = donors.rename(
    columns={
        'CustomerCustomFieldsRecurrent' : 'is_recurrent',
        'CustomerCustomFieldsVolunteer' : 'is_volunteer',
        'CustomerSex' : 'sex',
        'CustomerAreaIdsExternalId' : 'region_id',
        'CustomerAreaName' : 'region_name',
        'CustomerIanaTimeZone' : 'timezone',
        'CustomerTimeZoneSource' : 'timezone_src',
        'CustomerIdsMindboxId' : 'user_id',
        'CustomerIsEmailInvalid' : 'is_valid_email',
        'CustomerChangeDateTimeUtc' : 'update_date',
        'CustomerCustomerSubscriptionsDobroaifIsSubscribed' : 'subscribed',
        'CustomerCustomerSubscriptionsDobroaifSmsIsSubscribed' : 'subscribed_sms',
        'CustomerCustomerSubscriptionsDobroaifEmailIsSubscribed' : 'subscribed_email',
        'CustomerCustomerSubscriptionsDobroaifViberIsSubscribed' : 'subscribed_viber',
        'CustomerCustomerSubscriptionsDobroaifMobilePushIsSubscribed' : 'subscribed_mobile',
        'CustomerCustomerSubscriptionsDobroaifWebPushIsSubscribed' : 'subscribed_push',
    }
)


In [ ]:
orders = orders.rename(
    columns={
        'OrderIdsMindboxId' : 'order_id',
        'OrderFirstActionIdsMindboxId' : 'action_id',
        'OrderFirstActionDateTimeUtc' : 'create_date',
        'OrderFirstActionChannelIdsMindboxId' : 'contact_id',
        'OrderFirstActionChannelIdsExternalId' : 'contact_ext',
        'OrderFirstActionChannelName' : 'contact_name',
        'OrderAreaIdsExternalId' : 'region_id',
        'OrderTransactionIdsExternalId' : 'transaction_id',
        'OrderTotalPrice' : 'total_price',
        'OrderIdsWebsiteID' : 'website_id',
        'OrderCustomFieldsNewyear' : 'is_newyear',
        'OrderCustomFieldsNextPayDate' : 'next_pay_date',
        'OrderCustomFieldsRecurrent' : 'is_recurrent',
        'OrderCustomFieldsRepayment' : 'is_repayment',
        'OrderLineProductIdsWebsite' : 'product_id',
        'OrderLineProductName' : 'product_name',
        'OrderLineQuantity' : 'quantity',
        'OrderLineBasePricePerItem' : 'price_per_item',
        'OrderLinePriceOfLine' : 'price',
        'OrderLineStatusIdsExternalId' : 'status',
        'OrderCustomerIdsMindboxId' : 'user_id',
    }
)


In [ ]:
imports = imports.rename(
    columns={
        'CustomerActionCustomerIdsMindboxId' : 'user_id'
    }
)


### Преобразование дат и добавление новых столбцов

In [ ]:
donors['update_date'] = pd.to_datetime(donors['update_date'], dayfirst=True) + pd.Timedelta(hours=3)

donors['date'] = pd.to_datetime(donors['update_date'].dt.strftime('%Y-%m-%d'))
donors['month'] = pd.to_datetime(donors['update_date'].dt.strftime('%Y-%m-01'))
donors['year'] = pd.to_datetime(donors['update_date']).dt.year


In [ ]:
orders['create_date'] = pd.to_datetime(orders['create_date'], dayfirst=True) + pd.Timedelta(hours=3)

orders['date'] = pd.to_datetime(orders['create_date'].dt.strftime('%Y-%m-%d'))
orders['month'] = pd.to_datetime(orders['create_date'].dt.strftime('%Y-%m-01'))
orders['year'] = pd.to_datetime(orders['create_date']).dt.year

orders['day_of_week'] = pd.to_datetime(orders['create_date']).dt.day_name()
orders['part_of_day'] = orders['create_date'].dt.hour.apply(get_part_of_day)
orders['start_of_week'] = orders['date'] - pd.to_timedelta(orders['date'].dt.dayofweek, unit='d')

orders['cohort'] = orders.groupby('user_id')['create_date'].transform('min').dt.to_period('M')
orders['order_month'] = orders['date'].dt.to_period('M')



In [ ]:
channels['month'] = pd.to_datetime(channels['action_date'].dt.strftime('%Y-%m-01'))
channels['year'] = pd.to_datetime(channels['action_date']).dt.year
channels['event_dt'] = pd.to_datetime(channels['action_date'].astype(str) + ' ' + channels['action_time'].astype(str).str.zfill(2) + ':00:00')


### Работа с пропусками

#### donors.csv

In [ ]:
df_isna_info(donors)

In [ ]:
donors['is_recurrent'] = donors['is_recurrent'].fillna(False)

In [ ]:
donors['is_volunteer'] = donors['is_volunteer'].fillna(False)

In [ ]:
donors['sex'] = donors['sex'].apply(replace_sex)

In [ ]:
donors['is_valid_email'] = donors['is_valid_email'].fillna(False)

In [ ]:
donors['timezone'] = donors['timezone'].fillna('Неизвестно')
donors['timezone_src'] = donors['timezone_src'].fillna('Неизвестно')

In [ ]:
donors['region_id'] = donors['region_id'].fillna(0)
donors['region_name'] = donors['region_name'].fillna('Неизвестно')

In [ ]:
donors['subscribed'] = donors['subscribed'].fillna(False)
donors['subscribed_sms'] = donors['subscribed_sms'].fillna(False)
donors['subscribed_email'] = donors['subscribed_email'].fillna(False)
donors['subscribed_viber'] = donors['subscribed_viber'].fillna(False)
donors['subscribed_mobile'] = donors['subscribed_mobile'].fillna(False)
donors['subscribed_push'] = donors['subscribed_push'].fillna(False)

In [ ]:
df_isna_info(donors)

#### orders.csv

In [ ]:
df_isna_info(orders)

In [ ]:
orders['is_newyear'] = orders['is_newyear'].fillna(False)

In [ ]:
orders['is_recurrent'] = orders['is_recurrent'].fillna(False)

In [ ]:
orders['is_repayment'] = orders['is_repayment'].fillna(False)

In [ ]:
orders['product_name'] = orders['product_name'].fillna('Неизвестно')


In [ ]:
df_isna_info(orders)

#### channels.pkl

In [ ]:
df_isna_info(channels)

In [ ]:
channels['utm_campaign'] = channels['utm_campaign'].fillna('')
channels['utm_source'] = channels['utm_source'].fillna('')
channels['utm_medium'] = channels['utm_medium'].fillna('')


In [ ]:
df_isna_info(channels)

### Удаление столбцов

In [ ]:
orders['quantity'].value_counts()

In [ ]:
orders[['contact_id', 'contact_ext', 'contact_name']].value_counts()

In [ ]:
donors = donors.drop(
    [
        'region_id', 
        'timezone_src'
    ], 
    axis=1
)


In [ ]:
orders = orders.drop(
    [
        'region_id', 
        'transaction_id', 
        'next_pay_date', 
        'quantity', 
        'contact_id', 
        'contact_ext', 
        'price_per_item', 
        'price', 
        'action_id', 
        'website_id', 
        'product_id'
    ], 
    axis=1
)


In [ ]:
channels = channels.drop(
    [
        'action_time'
    ], 
    axis=1
)


In [ ]:
imports = imports[['user_id']]

### Преобразование типов данных

In [ ]:
df_column_astype(
    donors, 
    {
        'is_recurrent' : 'bool',
        'is_volunteer' : 'bool',
        'is_valid_email' : 'bool',
    }
)

In [ ]:
df_column_astype(
    orders, 
    {
        'total_price' : 'float64',
        'is_newyear' : 'bool',
        'is_recurrent' : 'bool',
        'is_repayment' : 'bool',
    }
)

### Преобразование назначения платежа и временной зоны

In [ ]:
orders['product_name'] = orders['product_name'].str.replace(r'Пожертвование\s+Благотворительный\s+фонд\s+«АиФ\.\s+Доброе сердце»\s*', '', regex=True)
orders['product_name'] = orders['product_name'].str.replace(r'\s*БФ\s+«АиФ\.\s+Доброе сердце»\s*', '', regex=True)
orders['product_name'] = orders['product_name'].str.replace(r'\s*\(хочу\s+помочь\)\s*', '', regex=True)
orders['product_name'] = orders['product_name'].str.replace(r'^Пожертвование\s+на\s+уставную\s+деятельность\s*$', 'На уставную деятельность', regex=True)

donors['timezone'] = donors['timezone'].str.replace(r'^.*\/', '', regex=True)


### Работа с дубликатами

In [ ]:
df_duplicated_info(donors)

In [ ]:
df_duplicated_info(orders)

In [ ]:
df_duplicated_info(channels)

In [ ]:
channels.shape[0]

In [ ]:
channels = channels.drop_duplicates()
channels.shape[0]

In [ ]:
df_duplicated_info(imports)

In [ ]:
imports.shape[0]

In [ ]:
imports = imports.drop_duplicates()
imports.shape[0]

### Переформирование индексов

In [ ]:
donors = donors.sort_values(by=['user_id', 'update_date']).reset_index(drop=True)

In [ ]:
orders = orders.sort_values(by=['user_id', 'create_date']).reset_index(drop=True)

In [ ]:
channels = channels.sort_values(by=['user_id', 'event_dt']).reset_index(drop=True)

In [ ]:
imports = imports.sort_values(by='user_id').reset_index(drop=True)

### Результат предобработки данных

In [ ]:
df_analysis(donors)

In [ ]:
df_analysis(orders)

In [ ]:
df_analysis(imports)

In [ ]:
df_analysis(channels)

### Вывод

Предобработка завершена. Данные предоставлены в хорошем качестве.

Для удобства работы:
- переменованы столбцы;
- подкорректирован тип данных, согласно заявленным значениям;
- добавлены новые столбцы с датами;
- обработаны пропуски, для корректной работы pandas;
- удалены лишние столбцы;
- удалены дубликаты;
- переформированы индексы.


## Дополнительные срезы, поля и постоянные переменные

In [ ]:
history_import_ids = imports['user_id'].unique()

In [ ]:
idx_first = orders.groupby('user_id').head(1).index

orders['is_first'] = False
orders.loc[idx_first, 'is_first'] = True


In [ ]:
first_payed_order_ids = orders.query('status == "Paid"').groupby('user_id')['create_date'].idxmin()

orders['is_first_payed'] = False
orders.loc[first_payed_order_ids, 'is_first_payed'] = True

In [ ]:
first_rec_payed_order_ids = orders.query('status == "Paid" and is_recurrent').groupby('user_id')['create_date'].idxmin()

orders['is_first_rec_payed'] = False
orders.loc[first_rec_payed_order_ids, 'is_first_rec_payed'] = True

In [ ]:
idx_first = channels.groupby('user_id').head(1).index

channels['is_first'] = False
channels.loc[idx_first, 'is_first'] = True


In [ ]:
orders['campaing'] = orders['product_name'].apply(put_campaing_name)

orders_payed = orders.query('status == "Paid"')

In [ ]:
user_cnt = donors['user_id'].nunique()

In [ ]:
rec_user_cnt = donors.query('is_recurrent')['user_id'].nunique()

In [ ]:
user_p_cnt = orders_payed['user_id'].nunique()

In [ ]:
rec_user_p_cnt = orders_payed.query('is_recurrent')['user_id'].nunique()


In [ ]:
donors['is_import'] = donors['user_id'].isin(imports['user_id'])
donors['is_active'] = donors['user_id'].isin(channels.query('year == 2024')['user_id'])
donors['is_payed'] = donors['user_id'].isin(orders.query('year == 2024')['user_id'])


## Пользовательские метрики

### Общий портрет пользователя

In [ ]:
message = f"""
    Количество уникальных пользователей: {donors['user_id'].nunique()}:

        - уникальных пользователей "исторического импорта": {imports.shape[0]};
        - уникальных пользователей "исторического импорта" в таблице с донорами: {donors.query('is_import')['user_id'].nunique()};
        - "новых" уникальных пользователей: {donors.query('is_import == False')['user_id'].nunique()};
        - уникальных пользователей, активных в 2024 году: {donors.query('is_active')['user_id'].nunique()};
        - уникальных пользователей, совершивших платежи в 2024 году: {donors.query('is_payed')['user_id'].nunique()}.

    Дата регистрации / редактирования c {donors['update_date'].min().date()} по {donors['update_date'].max().date()}'.

"""
 
print(message)

In [ ]:
draw_pie(
    donors['is_recurrent'].value_counts().reset_index(name='cnt'),
    'is_recurrent',
    'cnt',
    'Количество пользователей - рекурентов'
)

Рекурентных доноров только 3%.

In [ ]:
draw_pie_compare(
    donors.query('is_recurrent')['is_volunteer'].value_counts().reset_index(name='cnt'), 
    donors.query('not is_recurrent')['is_volunteer'].value_counts().reset_index(name='cnt'), 
    'is_volunteer', 
    'cnt', 
    'Количество пользователей - волонтеров',
    'Рекурент',
    'Не рекурент'
)

Среди рекурентов нет ни одного волонтера (всего их 6).

In [ ]:
draw_pie_compare(
    donors.query('is_recurrent')['sex'].value_counts().reset_index(name='cnt'), 
    donors.query('not is_recurrent')['sex'].value_counts().reset_index(name='cnt'), 
    'sex', 
    'cnt', 
    'Пол пользователя',
    'Рекурент',
    'Не рекурент'
)

Женщины чаще всего совершают пожертвования - более 60%, 20-30% в зависимости от рекурентности - мужчины, остальные доноры при регистрации не указали пол.

In [ ]:
draw_pie_compare(
    donors.query('is_recurrent')['is_valid_email'].value_counts().reset_index(name='cnt'), 
    donors.query('not is_recurrent')['is_valid_email'].value_counts().reset_index(name='cnt'), 
    'is_valid_email', 
    'cnt', 
    'Количество пользователей с валидной почтой',
    'Рекурент',
    'Не рекурент'
)

Менее 10% пользователей подтвердили свою электронную почту.

In [ ]:
donors_timezone = (
    donors
    .pivot_table(index='timezone', columns='is_recurrent', values='user_id', aggfunc='count', fill_value=0)
    .rename(columns={
        True: 'yes',
        False: 'no',
    })
    .sort_values(by='yes', ascending=False)
)

draw_bar_compare(
    donors_timezone.div(donors_timezone.sum(axis=0), axis=1).reset_index().head(10), 
    'timezone', 
    'no', 
    'yes', 
    'ТОП-10 часовых поясов', 
    'Не рекурент', 
    'Рекурент', 
    'Часовой пояс', 
    'Доля от общего количества'
)

У рекурентных пользователей чаще всего указан часовой пояс Москва (почти 45%), для нерекурентных чаще всего часовой пояс не указан (более 50%).

In [ ]:
donors_region = (
    donors
    .pivot_table(index='region_name', columns='is_recurrent', values='user_id', aggfunc='count', fill_value=0)
    .rename(columns={
        True: 'yes',
        False: 'no',
    })
    .sort_values(by='yes', ascending=False)
)

draw_bar_compare(
    donors_region.div(donors_region.sum(axis=0), axis=1).reset_index().head(10), 
    'region_name', 
    'no', 
    'yes', 
    'ТОП-10 регионов', 
    'Не рекурент', 
    'Рекурент', 
    'Регион', 
    'Доля от общего количества'
)

Для всех пользователей не характерно указание региона (больше 70%). У оставшихся доноров чаще всего - Vienna, возможно, это связано с использованием VPN.

In [ ]:
draw_plot(
    donors.query('is_recurrent')['month'].value_counts().reset_index(name='cnt').sort_values(by='month').set_index('month'), 
    'Дата появления/изменения рекурентных пользователей', 
    'Дата', 
    'Количество пользователей'
)

In [ ]:
draw_plot(
    donors['month'].value_counts().reset_index(name='cnt').sort_values(by='month').set_index('month'), 
    'Даты появления/изменения пользователей', 
    'Дата', 
    'Количество'
)

In [ ]:
draw_bar_compare(
    pd.pivot_table(
        (
            (
                donors
                .query('is_recurrent == True')[[
                    'subscribed', 
                    'subscribed_sms',
                    'subscribed_email',
                    'subscribed_viber',
                    'subscribed_mobile',
                    'subscribed_push',
                ]]
            )
            .melt(var_name='subscription_type', value_name='status')
        ), 
        index='subscription_type', 
        columns='status', 
        aggfunc=len, 
        fill_value=0
    )
    .rename(columns={
        True: 'yes',
        False: 'no',
    })
    .reset_index(), 
    'subscription_type', 
    'no', 
    'yes', 
    'Статистика по подпискам рекурентов', 
    'Нет подписки', 
    'Есть подписка', 
    'Тип подписки', 
    'Количество'
)

In [ ]:
draw_bar_compare(
    pd.pivot_table(
        (
            (
                donors[[
                    'subscribed', 
                    'subscribed_sms',
                    'subscribed_email',
                    'subscribed_viber',
                    'subscribed_mobile',
                    'subscribed_push',
                ]]
            )
            .melt(var_name='subscription_type', value_name='status')
        ), 
        index='subscription_type', 
        columns='status', 
        aggfunc=len, 
        fill_value=0
    )
    .rename(columns={
        True: 'yes',
        False: 'no',
    })
    .reset_index(), 
    'subscription_type', 
    'no', 
    'yes', 
    'Статистика по подпискам всех пользователей', 
    'Нет подписки', 
    'Есть подписка', 
    'Тип подписки', 
    'Количество'
)

- Уникальных пользователей: 23820.
- Уникальных пользователей «исторического импорта»: 7343.
- Уникальных «новых» пользователей: 16477:
- Рекуренты: 2.7% от общего количества.
- Волонтеры: менее 1% от общего количества.
- Пол: около 60% женщины, около 20-30% мужчины (варьируется в зависимости от рекурентности пользователя), у остальных 10-20% не определено.
- Меньше чем у 10% пользователей подтверждена  электронная почта.
- Тайм зона у большего пользователей не определена, или Moscow.
- Регион так же не определён в большинстве случаев, но потом идет Vienna (возможно, это связано с использованием  VPN). На третьем месте – Москва.
- Почти 99% пользователей подписаны только через email.

### "Исторический импорт"

In [ ]:
funnel_import = pd.DataFrame(
    {
        'type': [
            'Всего', 
            'Активные в 2024', 
            'Платящие в 2024',
            'Рекуренты в 2024'
        ], 
        'cnt': [
            donors.query('is_import == True')['user_id'].nunique(),
            channels.query('user_id in @history_import_ids and year == 2024')['user_id'].nunique(),
            orders_payed.query('user_id in @history_import_ids and year == 2024')['user_id'].nunique(),
            orders_payed.query('user_id in @history_import_ids and year == 2024 and is_recurrent == True')['user_id'].nunique()
        ]
    }
)



In [ ]:
draw_funnel(
    funnel_import, 
    'cnt', 
    'type', 
    'Воронка по историческому импорту'
)

Почти 75% пользователей из "исторического импорта" активны в 2024 году, но только 8% совершают пожертвования, 3% из которых рекуренты.

In [ ]:
draw_pie_compare(
    donors.query('is_active and is_import')['is_recurrent'].value_counts().reset_index(name='cnt'), 
    donors.query('not is_active and is_import')['is_recurrent'].value_counts().reset_index(name='cnt'), 
    'is_recurrent', 
    'cnt', 
    'Количество рекурентов',
    'Активные',
    'Не активные'
)

Среди активных пользователей почти 6% рекуренты, среди неактивных пользователей рекурентов было меньше 1%.

In [ ]:
draw_pie_compare(
    donors.query('is_active and is_import')['is_volunteer'].value_counts().reset_index(name='cnt'), 
    donors.query('not is_active and is_import')['is_volunteer'].value_counts().reset_index(name='cnt'), 
    'is_volunteer', 
    'cnt', 
    'Количество волонтеров',
    'Активные',
    'Не активные'
)

Среди пользователей "исторического импорта" нет ни одного волонтера.

In [ ]:
draw_pie_compare(
    donors.query('is_active and is_import')['sex'].value_counts().reset_index(name='cnt'), 
    donors.query('not is_active and is_import')['sex'].value_counts().reset_index(name='cnt'), 
    'sex', 
    'cnt', 
    'Пол пользователя',
    'Активные',
    'Не активные'
)

Пользователями "исторического импорта" чаще всего являются женщины, в 20% это мужчины, и около 30-35% доноров не указали свой пол, возможно, это какие-либо компании.

### "Новые" пользователи

In [ ]:
first_stat = pd.merge(
    orders.query('is_first')[['user_id', 'create_date']], 
    channels.query('is_first')[['user_id', 'event_dt']], 
    on='user_id',
    how='outer'
)

first_stat['days'] = (first_stat['create_date'] - first_stat['event_dt']).dt.days



In [ ]:
draw_box(
    first_stat, 
    'days', 
    'Кол-во дней между первой оплатой и первой активностью',
    10
)

Чаще всего между оплатой и первой активностью проходит не больше 1 дня. Причем сначала происходит пожертвование, после чего пользователь регистрируется в системе, тем самым совершает первую актиность на сайте. Но есть пользователи, которые ни разу не совершили платеж (причем не важно прошел он или нет), но активности по ним есть, т.е. он был зарегистрирован минуя оплату.

In [ ]:
df_isna_info(first_stat)

Почти 10% пользователей зарегистрированы в системе, минуя оплату.

In [ ]:
user_without_payed = first_stat[first_stat['create_date'].isna()]['user_id'].unique()

donors.query('user_id in @user_without_payed')['is_import'].value_counts()

Почти 70% это пользователи из "исторического импорта".

In [ ]:
old_donor_idx = donors.query('is_import == True')['user_id'].unique()
new_donor_idx = donors.query('is_import == False')['user_id'].unique()


Нашли уникальные ID пользователей из "исторического импорта", и "новых" доноров.

In [ ]:
first_channel = channels.query('is_first and user_id in @new_donor_idx')['channel_id'].value_counts().reset_index(name='user_cnt')
first_channel['channel_name'] = first_channel['channel_id'].map(dic_channels)

draw_bar_horizontal(
    first_channel, 
    'channel_name',
    'user_cnt',
    'Каналы первой активности "новых" пользователей',
    'Канал',
    'Количество пользователей',
)

Чаще всего каналом для первой активности "новых" пользователей указан `Сайт` (более чем в 50%, у 17% он определен как `Email`).

In [ ]:
draw_plot(
    channels.query('is_first and user_id in @new_donor_idx')['month'].value_counts().reset_index(name='cnt').sort_values(by='month').set_index('month'), 
    'Первые активности "новых" пользователей по месяцам', 
    'Дата', 
    'Количество'
)

- 75% пользователей «исторического импорта» проявляют активность, при этом платящих клиентов только 8.6%, рекуренты из них – 3%.
- Волонтеров среди «исторического импорта» нет.
- Первым касанием с пользователем чаще всего является первая оплата (или ее попытка), во время которой фонд получает контактные данные пользователя, и может в дальнейшем информировать его с помощью рассылок. Но для 10% есть активности, хотя не было никаких оплат (треть из них это «исторический импорт»).
- Первая активность «новых пользователей» чаще всего была через Сайт, или Email.
- Всплеск первых активностей новых пользователей наблюдается августе 2022 года, феврале 2024 и сентябре 2023.
- Всплеск первых оплат новых пользователей наблюдался те же месяца.

### Оценка пользовательской активности

In [ ]:
pivot_dau = (
    orders_payed
    .pivot_table(index='date', values='user_id', aggfunc='nunique', fill_value=0)
    .reset_index()
    .set_index('date')
)


In [ ]:
draw_plot(
    pivot_dau, 
    'DAU', 
    'Дата', 
    'Количество уникальных пользователей'
)

In [ ]:
pivot_wau = (
    orders_payed
    .pivot_table(index='start_of_week', values='user_id', aggfunc='nunique', fill_value=0)
    .reset_index()
    .set_index('start_of_week')
)


In [ ]:
draw_plot(
    pivot_wau, 
    'WAU', 
    'Дата', 
    'Количество уникальных пользователей'
)

In [ ]:
pivot_mau = (
    orders_payed
    .pivot_table(index='month', values='user_id', aggfunc='nunique', fill_value=0)
    .reset_index()
    .set_index('month')
)


In [ ]:
draw_plot(
    pivot_mau, 
    'MAU', 
    'Дата', 
    'Количество уникальных пользователей'
)

На основе анализа графиков DAU (Daily Active Users), WAU (Weekly Active Users) и MAU (Monthly Active Users) можно сделать следующие выводы:
- Все три показателя демонстрируют общую тенденцию к росту, что указывает на стабильное увеличение пользовательской базы и вовлеченности.
- Наблюдаются локальные падения активности в марте 2022 и феврале 2023 годов, что может быть связано с сезонными факторами или внешними событиями.
- В конце 2021 и середине 2022 года отмечен значительный рост, вероятно, вызванный успешными маркетинговыми кампаниями или улучшениями в сервисе.
- Максимальный пик активности наблюдается в феврале 2024 года, что может свидетельствовать о крупных обновлениях или эффективных стратегиях привлечения пользователей.

## RFM анализ - сегментация клиентской базы

RFM — это метод, используемый для анализа потребительской ценности.

Он группирует клиентов на основе истории их транзакций:

- `Recency/Давность` — как давно клиент совершил покупку;
- `Frequency/Частота` — как часто он совершал покупки;
- `Monetary Value/Денежная ценность` — сколько он тратил.
  
В нашем случае выберем в качестве:

- `Recency` — время отсутствия пользователя (количество дней с момента последнего пожертвования);
- `Frequency` — количество пожертвований в день;
- `Monetary Value` — суммарный доход от пожертвований пользователя.


In [ ]:
orders['order_id'].nunique()

In [ ]:
draw_pie(
    orders['status'].value_counts().reset_index(),
    'status',
    'count', 
    'Статус оплаты пожертвования'
)

Для RFM анализа будем использовать только такие платежи, которые имеют статус `Paid`.

In [ ]:
info_from_orders = (
    orders_payed
    .groupby('user_id')
    .agg(
        {
            'create_date': lambda x: (start_date - x.max()).days, 
            'order_id': 'count',
            'total_price' : 'sum',
        }
    )
    .rename(
        columns={
            'create_date': 'r', 
            'order_id': 'order_cnt',
            'total_price' : 'm'
        }
    )
    .reset_index()
)


Для `Recency` необходимо посчитать количество пожертвований в день, для каждого пользователя, поэтому необходимо найти суммарное время активности пользователя в фонде.

In [ ]:
info_from_active = (
    channels
    .groupby('user_id')
    .agg(
        {
            'action_date': lambda x: (x.max() - x.min()).days, 
        }
    )
    .rename(
        columns={
            'action_date' : 'active_day_cnt'
        }
    )
    .reset_index()
)


In [ ]:
rfm = info_from_orders.merge(info_from_active, how = 'left', on='user_id').fillna(0)
rfm['f'] = (rfm['order_cnt'] / (rfm['active_day_cnt'] + 1)).round(4) 
rfm = rfm[['user_id', 'r', 'f', 'm']]
rfm

Теперь необходимо просмегментировать каждого пользователя по каждому параметру рангом от 1 до 3. Для подсчета ранга будем использовать 33 и 66 перцентиль, при подсчете перцентилей выбросы учитывать не будем.

In [ ]:
draw_box(rfm, 'r', 'Диаграмма размаха для параметра "Давность"')

rfm['r_seg'] = pd.cut(rfm['r'], labels=[1,2,3], bins=rank_value(rfm, 'r'), include_lowest=True)


In [ ]:
draw_box(rfm, 'f', 'Диаграмма размаха для параметра "Частота"', 0.05)

rfm['f_seg'] = pd.cut(rfm['f'], labels=[3,2,1], bins=rank_value(rfm, 'f'), include_lowest=True)


In [ ]:
draw_box(rfm, 'm', 'Диаграмма размаха для параметра "Денежная ценность"', 3000)

rfm['m_seg'] = pd.cut(rfm['m'], labels=[3,2,1], bins=rank_value(rfm, 'm'), include_lowest=True)


In [ ]:
rfm['RFM'] = rfm['r_seg'].astype('str') + rfm['f_seg'].astype('str') + rfm['m_seg'].astype('str')
rfm

В итоге получилось 27 сегментов с числовыми значениями распределёнными определённым образом в интервале от 111 до 333 включительно, где: 
- первая цифра - показатель “Давность” пожертвования:
    - 1 - пожертвование не позднее 165 дней назад.
    - 2 - пожертвование от 165 до 563 дней назад.
    - 3 - пожертвование раньше 563 дней назад.
- вторая - показатель “Частота” (отношение количества пожертвований на количество дней жизни донора):
    - 1 - пожертвование не реже 0.0069 раза в сутки.
    - 2 - пожертвование от 0.0023 до 0.0069 раза в сутки.
    - 3 - пожертвование реже 0.0023 раза в сутки.
- третья - показатель “Денежная ценность”:
    - 1 - суммарно более 500 рублей.
    - 2 - суммарно от 300 до 500 рублей.
    - 3 - менее 300 рублей.

Для удобства и упрощения восприятия некоторые сегменты можно объединять в группы.


In [ ]:
rfm_data

In [ ]:
rfm_stat = (
    rfm['RFM']
    .value_counts()
)


In [ ]:
draw_bar_horizontal(
    rfm_stat.reset_index(name='cnt'), 
    'RFM',
    'cnt',
    'RFM - анализ',
    'Сегмент',
    'Количество пользователей'
)

In [ ]:
draw_bar_horizontal(
    (
        pd
        .merge(rfm_data, rfm_stat.reset_index(name='cnt'), on='RFM')
        .groupby('Определение группы')['cnt']
        .sum()
        .reset_index()
        
    ),
    'Определение группы',
    'cnt',
    'RFM - анализ по группам',
    'Группа',
    'Количество пользователей'
)

In [ ]:
rfm_lost = rfm.query('RFM in ("333", "331", "332", "333")')['user_id'].unique()
donors_lost = donors.query('user_id in @rfm_lost')


In [ ]:
draw_pie(
    donors_lost['is_import'].value_counts().reset_index(name='cnt'),
    'is_import',
    'cnt',
    'Количество "потерянных" пользователей из исторического импорта'
)

In [ ]:
draw_pie(
    donors_lost['is_recurrent'].value_counts().reset_index(name='cnt'), 
    'is_recurrent',
    'cnt',
    'Количество "потерянных" пользователей - рекурентов'
)

Анализ сегментов выявил наиболее популярные категории клиентов благотворительного фонда. Сегмент 111 (15.12%) приносит наибольшую ценность компании. Сегмент 333 (13.32%) - менее ценный для компании. Сегмент 113 (9.42%) - клиенты готовы жертвовать крупные суммы, но их необходимо мотивировать к более частым пожертвованиям. Сегмент 223 (8.99%) клиенты нейтральны и требуют более глубокого анализа.


Выводы:
- Удержание доноров из сегмента 111 (недавние, частые и крупные пожертвования) - наибольшая ценность для фонда:
    - Отправление индивидуальных писем с благодарностями и отчетами о результатах пожертвований.
    - Создание программ, которые поощряют постоянные пожертвования, например, предоставление донорских привилегий.
- Стимулирование доноров из сегмента 113 (недавние и частые, но малые пожертвования) - готовы жертвовать суммы, но нужно мотивировать на увеличение пожертвований:
    - Регулярное информирование таких доноров о текущих проектах и результатах их пожертвований.
    - Предложение поучаствовать в различных проектах фонда, которые могут их заинтересовать.
- Активизация доноров из сегмента 223 (средние значения по всем параметрам) - нейтральный сегмент, требует внимания и анализа:
    - Анализ обратной связи, чтобы понять их потребности и желания.
- Реактивация доноров из сегмента 333 (редкие, давние и небольшие пожертвования) - менее ценный сегмент, но потенциально можно реактивировать:
    - Предоставление уникальных предложений для возвращения доноров, такие как матчи пожертвований (удваивание пожертвований за счет крупного донора).
    - Использование автоматических напоминаний и уведомлений о возможности пожертвовать.
- Работа с другими сегментами:
    - Для сегментов с высокими показателями F и M, но низкими R (например, 211, 213): необходимо сосредоточиться на возвращении доноров, стимулируя частоту пожертвований.
    - Для сегментов с высокими показателями R и F, но низкими M (например, 321, 121): необходимо сосредоточиться на увеличении среднего размера пожертвования.


Общие рекомендации:

- Использование данных для создания персонализированных предложений и благодарностей.
- Постоянный анализ поведения доноров и обновление RFM-анализа.
- Прозрачная отчетность о том, как используются пожертвования.
- Вовлечение доноров в деятельность фонда.
  
Расширение базы точек контактов доноров может помочь вернуть "Спящих клиентов":
- Сбор номеров мобильных телефонов при регистрации или активном действии, объясняя это необходимостью более оперативного и удобного информирования. Электронные письма не самый удачный канал для оповещения доноров, но именно он чаще всего встречается у пользователей.
- Использование SMS-уведомлений с напоминаниями о пожертвованиях, ссылками на отчеты и новостями фонда.
- Оптимизация частоты и контента сообщений, сообщения должны быть краткими и информативными, необходимо изюегать чрезмерного количества отправок.
- Использование мессенджеров.
  
Стимулирование рекуррентных пожертвований:
- Объяснение значимости регулярных пожертвований.
- Простая настройка и отмена регулярных платежей.
- Идеи для стимулирования, такие как округление счетов (работа совместно с банками).
- Программы лояльности и поощрения.
- Прозрачность и отчетность.
- Мобильные приложения и автоматизация.
  
Эти шаги помогут эффективно предложить рекуррентные платежи донорам, которые готовы к этому, и обеспечат стабильное финансирование для реализации благотворительных проектов.

## Коммерческие метрики

### Средний чек

In [ ]:
draw_box(
    orders_payed,
    'total_price',
    'Средний оплаченный чек пользователя',
    1500
)

In [ ]:
draw_box(
    orders_payed.groupby('user_id')['order_id'].count().reset_index(name='cnt'),
    'cnt',
    'Количество оплаченных чеков пользователем',
    5
)

In [ ]:
draw_box(
    orders_payed.groupby('date')['order_id'].count().reset_index(name='cnt'),
    'cnt',
    'Количество оплаченных чеков в день',
    150
)

In [ ]:
orders_payed.pivot_table(index='month', values='total_price', aggfunc='mean')

In [ ]:
draw_plot(
    orders_payed.pivot_table(index='month', values='total_price', aggfunc='mean'), 
    'Динамика распределения среднего чека', 
    'Месяц', 
    'Средний чек'
)

In [ ]:
orders_payed.groupby('month')['total_price'].mean().reset_index(name='mean_price').sort_values(by='mean_price', ascending=False).head(5)

In [ ]:
orders_payed.groupby('month')['total_price'].mean().reset_index(name='mean_price').sort_values(by='mean_price', ascending=False).tail(5)

In [ ]:
draw_bar_horizontal(
    orders_payed.groupby('product_name')['total_price'].mean().reset_index(name='mean_price'), 
    'product_name', 
    'mean_price',
    'ТОП-10 назначений по среднему чеку',
    'Средний чек',
    'Назначение платежа',
    is_top=True
)

Выводы:

- Среднее пожертвование пользователя варьируется от 110 до 500 рублей.
- Среднее количество оплат на одного пользователя от 1 до 3х.
- Среднее количество платежей в день колеблется от 19 до 53.
- Самый высокий средний чек наблюдался в апреле 2024 года, в феврале 2023 и марте 2023 года.
- Самый низкий средний чек наблюдался в феврале 2022, январе 2024 и ноябре 2021 года.
- Самый высокий средний чек по назначению платежа был для Александра Слепухина, Дарьи Тортоевой и Никиты Яковлева.

### Главный источник входа для пользователей

In [ ]:
main_channel_stat = (
    channels
    .query('is_first')['channel_id']
    .value_counts()
    .reset_index(name='percent')
)
main_channel_stat['channel_name'] = main_channel_stat['channel_id'].map(dic_channels)

draw_bar_horizontal(
    main_channel_stat, 
    'channel_name', 
    'percent',
    'Статистика по первому источнику входа для всех пользователей',
    'Доля',
    'Канал привлечения'
)


In [ ]:
main_channel_stat_import =(
    channels
    .query('is_first and user_id in @history_import_ids')
    .pivot_table(index='month', columns='channel_id', values='user_id', aggfunc= 'count')
)

main_channel_stat_import.rename(columns=dic_channels, inplace=True)



In [ ]:
draw_plot(
    main_channel_stat_import, 
    'Динамика каналов привлечения для "исторического импорта"', 
    'Месяц', 
    'Количество доноров'
)

In [ ]:
main_channel_stat_import =(
    channels
    .query('is_first and user_id not in @history_import_ids')
    .pivot_table(index='month', columns='channel_id', values='user_id', aggfunc= 'count')
)

main_channel_stat_import.rename(columns=dic_channels, inplace=True)



In [ ]:
draw_plot(
    main_channel_stat_import, 
    'Динамика каналов привлечения для "новых" доноров', 
    'Месяц', 
    'Количество доноров'
)

In [ ]:
channels.query('is_first')['month'].value_counts().reset_index().head(5)

Главным источником входа пользователя чаще всего является Сайт. Клиент делает пожертвование, после кого оставляет свои контакты в системе, этот момент можно считать первым касанием с пользователем.

Пользователи из "исторического импорта" совершили свое касание после импорта данных до мая 2022, кто этого хотел, больше не было новых вхождений исторического импорта. 

Новые пользователи активно подключаются к фонду.

### Активность по дням недели

In [ ]:
draw_bar(
    orders_payed['day_of_week'].value_counts().reset_index(name='cnt'), 
    'day_of_week', 
    'cnt',
    'Количество оплаченных платежей по дням недели',
    'День недели',
    'Количество'
)

In [ ]:
orders_by_dayweek = orders_payed.groupby('day_of_week')['total_price'].sum().reset_index().sort_values(by='total_price', ascending=False)



In [ ]:
draw_bar(
    orders_by_dayweek, 
    'day_of_week', 
    'total_price',
    'Общая сумма всех оплаченных платежей по дням недели',
    'День недели',
    'Сумма'
)

### Активность по времени суток

In [ ]:
draw_pie(
    orders_payed['part_of_day'].map(dict_part_of_day).value_counts().reset_index(name='cnt'),
    'part_of_day',
    'cnt',
    'Количество оплаченных платежей по времени суток'
)

In [ ]:
orders_by_partday = orders_payed.groupby('part_of_day')['total_price'].sum().reset_index()
orders_by_partday.index = orders_by_partday.index.map(dict_part_of_day)

draw_pie(
    orders_by_partday,
    'part_of_day',
    'total_price',
    'Общая сумма всех оплаченных платежей по времени суток'
)

### Активность по месяцам

In [ ]:
draw_bar_horizontal(
    orders_payed['month'].value_counts().reset_index(name='cnt'), 
    'month', 
    'cnt',
    'ТОП-10 месяцев по количеству оплаченных платежей',
    'Количество платежей',
    'Месяц',
    is_top=True
)

In [ ]:
draw_bar_horizontal(
    orders_payed.groupby('month')['total_price'].sum().reset_index(name='cnt'), 
    'month', 
    'cnt',
    'ТОП-10 месяцев по суммарным сборам',
    'Рубли',
    'Месяц'
)

In [ ]:
draw_plot(
    channels['month'].value_counts().reset_index(name='cnt').sort_values(by='month').set_index('month'), 
    'Гистограмма распределения платежей', 
    'Дата', 
    'Количество'
)

In [ ]:
draw_plot(
    orders_payed['month'].value_counts().reset_index(name='cnt').sort_values(by='month').set_index('month'), 
    'Гистограмма распределения платежей', 
    'Дата', 
    'Количество'
)

Выводы:

- Самые активные дни недели по количеству пожертвований вторник, пятница. Менее активные суббота, воскресенье и понедельник.
- Чаще всего пожертвования происходят в ночное время суток, реже всего утром.
- Самые активные месяца за всю историю по количеству пожертвований – февраль 2024, декабрь 2023 и сентябрь 2023 года.

## Маркетинговые метрики

### Первое касание пользователя

In [ ]:
draw_bar_horizontal(
    channels.query('is_first')['user_action'].value_counts().reset_index(name='cnt'), 
    'user_action', 
    'cnt',
    'Статистика по виду действия во время первого входа пользователя',
    'Количество',
    'Название действия',
    is_top=True
)

In [ ]:
draw_bar_horizontal(
    channels.query('is_first and utm_campaign != ""')['utm_campaign'].value_counts().reset_index(name='cnt'), 
    'utm_campaign', 
    'cnt',
    'Статистика по кампаниям во время первого входа пользователя',
    'Количество',
    'Кампания',
    is_top=True
)

In [ ]:
draw_bar_horizontal(
    channels.query('is_first and utm_campaign != ""')['utm_source'].value_counts().reset_index(name='cnt'), 
    'utm_source', 
    'cnt',
    'Статистика по ресурсу во время первого входа пользователя',
    'Количество',
    'Ресурс',
    is_top=True
)


Первое касание пользователя чаще всего сопровождалось действием - Оплата заказа.

Первая активность клиента редко попадала в кампании, но если такое и случалось, то самым популярым ресурсом для первого входа можно назвать vk, самой популярной компанией при входе - demid-lebedev.

### Назначение платежа

In [ ]:
product_name = (
    orders_payed
    .groupby('product_name')
    .agg(
        {
            'create_date': lambda x: (x.max() - x.min()).days + 1, 
            'user_id' : 'nunique',
            'order_id' : 'count',
            'total_price' : 'sum',
        }
    )
    .rename(
        columns={
            'create_date' : 'days',
            'user_id' : 'user_cnt',
            'order_id' : 'order_cnt',
        }
    )
    .reset_index()
)

product_name_analysis = {
    'days' : 'ТОП-10 назначений по времени жизни',
    'user_cnt' : 'ТОП-10 назначений по количеству уникальных пользователей',
    'order_cnt' : 'ТОП-10 назначений по количеству оплаченных платежей',
    'total_price' : 'ТОП-10 назначений по суммарным пожертвованиям',
}

for analysis in product_name_analysis:
    draw_bar_horizontal(
        product_name, 
        'product_name', 
        analysis,
        product_name_analysis[analysis],
        'Количество',
        'Назначение платежа',
        is_top=True
    )    

### Кампании

In [ ]:
round(channels.query('utm_campaign != ""').shape[0]/channels.shape[0] * 100, 2)

In [ ]:
utm_campaign = (
    channels
    .query('utm_campaign != ""')
    .groupby('utm_campaign')
    .agg(
        {
            'action_date': lambda x: (x.max() - x.min()).days, 
            'user_id' : 'nunique',
            'user_action' : 'count'
        }
    )
    .rename(
        columns={
            'action_date' : 'days',
            'user_id' : 'user_cnt',
            'user_action' : 'action_cnt',
        }
    )
    .reset_index()
)

utm_campaign_analysis = {
    'days' : 'ТОП-10 кампаний по времени жизни',
    'user_cnt' : 'ТОП-10 кампаний по количеству уникальных пользователей',
    'action_cnt' : 'ТОП-10 кампаний по количеству активностей',
}

for analysis in utm_campaign_analysis:
    draw_bar_horizontal(
        utm_campaign, 
        'utm_campaign', 
        analysis,
        utm_campaign_analysis[analysis],
        'Количество',
        'Кампания',
        is_top=True
    )    

### Кампании по каналам

In [ ]:
channel_stat = channels['channel_id'].value_counts().reset_index(name='cnt')
channel_stat['channel_name'] = channel_stat['channel_id'].map(dic_channels)


In [ ]:
draw_bar_horizontal(
    channel_stat, 
    'channel_name', 
    'cnt',
    'Статистика событий по каналам',
    'Количество событий',
    'Канал'
)


In [ ]:
channels_not_empty = (
    channels.groupby('channel_id')['utm_campaign']
    .apply(lambda x: all(x == ''))
    .reset_index()
    .rename(columns={'utm_campaign': 'all_empty'})
)

channels_not_empty = channels_not_empty[channels_not_empty['all_empty'] == False]
channels_not_empty['channel_name'] = channels_not_empty['channel_id'].map(dic_channels)

print('Список каналов привлечения, использующих кампании:')
display(channels_not_empty)

Не на всех каналах бывают кампании, далее рассмотрим только такие.

In [ ]:
channels_not_empty_ids = channels_not_empty['channel_id'].unique()

for channel in dic_channels:

    if channel in channels_not_empty_ids:
        draw_bar_horizontal(
            (
                channels
                .query('channel_id == @channel')['utm_campaign']
                .value_counts()
                .reset_index(name='cnt')
            ), 
            'utm_campaign', 
            'cnt',
            'Канал привлечения - `{}`'.format(dic_channels[channel]),
            'Количество событий',
            'Кампания',
            is_top=True
    )
    

Составим список всех кампаний, которые были в фонде, подсчитаем для каждой дату начала и конца по активностям в таблице channels, количество уникальных пользователей, участвовавших в кампании.

In [ ]:
campaign_dates = channels.query('utm_campaign != ""').groupby('utm_campaign').agg(min_date=('action_date', 'min'), max_date=('action_date', 'max'), user_cnt=('user_id', 'nunique')).reset_index().sort_values(by='min_date')
campaign_dates['min_month'] = pd.to_datetime(campaign_dates['min_date'].dt.strftime('%Y-%m-01'))
campaign_dates['min_year'] = pd.to_datetime(campaign_dates['min_date']).dt.year

campaign_dates

Найдем кампании за 2024 год, ранее мы наблюдали всплеск активности в это время. Отсортируем кампании по количеству уникальных пользователей.

In [ ]:
campaign_2024 = campaign_dates[(campaign_dates['min_year'] == 2024)]

In [ ]:
campaign_2024.sort_values(by='user_cnt', ascending=False).head(10)

Найдем из таблицы пожертвований назначение платежей, они должны быть созвучны компаниям в таблице channels, найдем соотвествия, а так же каналы, которые активно привлекают пользователей совершать платеж в 2024 году.

In [ ]:
orders_2024 = orders_payed[(orders_payed['year'] == 2024)].groupby('campaing')['total_price'].sum().reset_index().sort_values(by='total_price', ascending=False).head(10)


In [ ]:

# Перебор каждой строки
for index, row in orders_2024.iterrows():
    campaing_name = row['campaing']
    if is_translit_name(campaing_name):
        tmp  = channels.query('utm_campaign == @campaing_name')['channel_id'].value_counts().reset_index()
        tmp['channel_name'] = tmp['channel_id'].map(dic_channels)
        if tmp.shape[0] > 0:
            print(f"Кампания: {campaing_name}, Общая сумма: {row['total_price']}")
            display(tmp[['channel_name', 'count']])

Чаще всего встречается канал "utm_term Не указан".

### Эффективность действий по каналам

Поскольку и оплаты и записи из таблицы channels являются своего рода активностями, соберем их в одну единую таблицу. Отсоритурем все активности по каждому пользователю, и попробуем найти, после каких собятий было событие внесения пожертвования. Найдем такое события и проанализируем их.

In [ ]:
order_event = orders_payed[['user_id', 'create_date', 'total_price']].rename(columns={'create_date': 'event_dt'})
order_event['channel_id'] = 0


In [ ]:
channel_event = channels[['user_id', 'event_dt', 'channel_id']]
channel_event['total_price'] = 0


In [ ]:
events = pd.concat([order_event, channel_event]).sort_values(by=['user_id', 'event_dt']).reset_index(drop=True)
events['next_channel_id'] = events.groupby('user_id')['channel_id'].shift(-1)
events['next_event_dt'] = events.groupby('user_id')['event_dt'].shift(-1)
events['next_total_price'] = events.groupby('user_id')['total_price'].shift(-1)
events

In [ ]:
channel_percentages = []

for channel in dic_channels:
    total_events = events[events['channel_id'] == channel].shape[0]
    if total_events > 0:
        percent = (events[(events['channel_id'] == channel) & (events['next_channel_id'] == 0)].shape[0] / total_events) * 100
        channel_percentages.append((dic_channels[channel], percent))

channel_percentages.sort(key=lambda x: x[1], reverse=True)


Самым эффективным каналом привлечения является канал `ida`, почти после каждой четвертой активности есть запись по оплате пожертвования. Выше в лидерах фигурировало назначение платежа egor_czukanov. Одним из каналов привлечения как был был канал `Ida`. Видимо реклама у известных блогеров очень эффективна в привлечении новых пользователей. Но поскольку данных по затратам у нас нет, сложно сказать про окупаемость этих платежей.

Канал привлечения email очень неэффективен, только 0.66% событий заканчиваются платежем.

Канал `utm_temn Не указан` занимает положение в середине рейтинга, больше 5% событий завершаются платежем. Выше мы анализировали всплеск активности в начале 2024 года, и там чаще всего встречался этот канал привлечения.

In [ ]:
draw_bar_horizontal(
    pd.DataFrame(channel_percentages).rename(columns={0: 'channel', 1: '%'}), 
    'channel', 
    '%',
    'Процент событий, после которых шло пожертвование',
    'Процент от всех событий по каналу',
    'Канал'
)

## Когортный анализ

In [ ]:
df_cohort = orders_payed.groupby(['cohort', 'order_month']).agg(cnt=('user_id', 'nunique'), sum_price=('total_price', 'sum')).reset_index(drop=False)
df_cohort['lifetime'] = (df_cohort.order_month - df_cohort.cohort).apply(attrgetter('n'))


In [ ]:
last_suitable_acquisition_date = observation_date - relativedelta(months=horizon_months)
cohort_end = pd.Timestamp(last_suitable_acquisition_date).to_period('M')

first_suitable_acquisition_date = observation_date - relativedelta(months=horizon_months * 2.5)
cohort_start = pd.Timestamp(first_suitable_acquisition_date).to_period('M')

df_cohort = df_cohort[(df_cohort['cohort'] >= cohort_start) & (df_cohort['cohort'] <= cohort_end)]


In [ ]:
# Рассчитываем размер когорты
cohort_sizes = df_cohort[df_cohort['lifetime'] == 0][['cohort', 'cnt']].set_index('cohort').rename(columns={'cnt': 'cohort_size'})

# Добавляем размер когорты к основным данным
df_cohort = df_cohort.merge(cohort_sizes, on='cohort')

df_cohort.head()

### Retention Rate - удержание клиентов

In [ ]:
cohort_pivot = df_cohort.pivot_table(index='cohort', columns='lifetime', values='cnt')

In [ ]:
retention_matrix = cohort_pivot.divide(cohort_pivot.iloc[:, 0], axis=0)


In [ ]:
draw_heatmap(
    retention_matrix.iloc[:, 1:14],
    'Тепловая карта удержания', 
    'Лайфтайм', 
    'Месяц',
)


In [ ]:
draw_T_plot(
    retention_matrix.iloc[:, 1:14],
    'Кривые удержания по дням привлечения',
    'Лайфтайм',
    'Коэффициент удержания'
)

### LTV - пожизненная ценность клиента

In [ ]:
# Рассчитываем накопительную сумму продаж
df_cohort['cumulative_sum_price'] = df_cohort.groupby('cohort')['sum_price'].cumsum()

# Рассчитываем LTV, деля накопительную сумму на размер когорты
df_cohort['ltv'] = df_cohort['cumulative_sum_price'] / df_cohort['cohort_size']

# Создаем сводную таблицу для LTV
ltv_matrix = df_cohort.pivot_table(index='cohort', columns='lifetime', values='ltv', aggfunc='sum').fillna(0)

display(ltv_matrix)

In [ ]:
draw_T_plot(
    ltv_matrix.iloc[:, 0:14],
    'Кривые LTV по дням привлечения',
    'Лайфтайм',
    'LTV'
)

### Conversion - переход от разового платежа в рекуренты

In [ ]:
print('Зарегистрировано пользователей: {} ({} - заявлены как рекуренты).'.format(user_cnt, rec_user_cnt))

In [ ]:
print('Хотя бы раз совершивших пожертвование пользователей: {} ({} - делали хоть раз рекурентные платежи).'.format(user_p_cnt, rec_user_p_cnt))

In [ ]:
conversion_raw = (
    pd.merge(
        orders_payed.query('is_first_payed')[['user_id', 'create_date']],
        orders_payed.query('is_first_rec_payed')[['user_id', 'create_date']],
        on='user_id',
        how='left'
    ).rename(
        columns={
            'create_date_x' : 'is_one',
            'create_date_y' : 'is_rec'
        }
    )
)

conversion_raw['lifetime'] = conversion_raw.apply(
    lambda row: (row['is_rec'].to_period('M') - row['is_one'].to_period('M')).n if pd.notna(row['is_rec']) else None,
    axis=1
)
conversion_raw.head()

In [ ]:
draw_box(
    conversion_raw,
    'lifetime',
    'Диаграмма размаха периода перехода от первого разового платежа к первому регулярному, мес.',
)

In [ ]:
conversion = conversion_raw.pivot_table(index=None, columns='lifetime', values='user_id', aggfunc='count', fill_value=0)

cumsum = conversion.cumsum(axis=1)

conversion.loc['cumsum'] = cumsum.iloc[-1]
conversion.loc['normal'] = conversion.loc['cumsum'] / user_p_cnt
conversion

In [ ]:
draw_heatmap(
    conversion.iloc[:, 1:14].loc[['normal']], 
    'Тепловая карта конверсии платящих пользователей в рекуренты', 
    'Лайфтайм', 
    'Дата',
    annot=True, 
    size_x=12, 
    size_y=1
)

- Наибольшее удержание по оплатам в последующие периоды наблюдается для пользователей, совершивших первый платеж в июле 2021 года. Эти клиенты показывали удержание до 27% на четвертый месяц «жизни».
- Наименьшее удержание у пользователей в феврале, мае и августе 2022 года.
- Конверсия платящих пользователей в рекуренты очень маленькая, за год только 3.6% пользователей перешли к регулярному платежу.
- В среднем от первого платежа до первого регулярного платежа проходит от 0 до 15 месяцев.
- Самый высокий коэффициент LTV у пользователей, совершивших первый платеж в июле и апреле 2021 года.
- Самый низкий LTV у пользователей, впервые совершивших платеж в феврале 2022 года.

## Проверка гипотез

### Гипотеза: платящие пользователи из "исторического импорта" тратят больше денег на пожертвования

**Нулевая гипотеза:** средние значения пожертвований пользователей из двух групп равны.

**Альтернативная гипотеза:** среднее значение пожертвований пользователей из "исторического импорта" больше, чем среднее значение пожертвований "новых" пользователей.

In [ ]:
check_ttest_ind(
    orders_payed.query('user_id in @history_import_ids '), 
    orders_payed.query('user_id not in @history_import_ids '), 
    'total_price', 
    'greater'
)

**Вывод:** Полученное значение p-value значительно больше заданного уровня значимости, поэтому нет оснований отвергать нулевую гипотезу. Статистически значимых различий между средними значениями пожертвований пользователей в двух группах нет.


In [ ]:
print(
    'Средний чек пожертвований для "исторического импорта": {:.2f}'
    .format(orders_payed.query('user_id in @history_import_ids ')['total_price'].mean())
)

In [ ]:
print(
    'Средний чек пожертвований для "новых" пользователей: {:.2f}'
    .format(orders_payed.query('user_id not in @history_import_ids ')['total_price'].mean())
)


### Гипотеза: регулярные пожертвования меньше разовых

**Нулевая гипотеза:** средние значения пожертвований из двух групп равны.

**Альтернативная гипотеза:** среднее значение регулярных пожертвований меньше разовых пожертвований.

In [ ]:
check_ttest_ind(
    orders_payed.query('is_recurrent == True'), 
    orders_payed.query('is_recurrent == False'), 
    'total_price', 
    'less'
)

**Вывод:** Полученное значение p-value значительно меньше заданного уровня значимости, поэтому основания отвергать нулевую гипотезу. Статистически значимые различия между средними значениями пожертвований - есть, регулярные платежи меньше разовых.

In [ ]:
print(
    'Средний чек регулярного платежа: {:.2f}'
    .format(orders_payed.query('is_recurrent == True')['total_price'].mean())
)

In [ ]:
print(
    'Средний чек разового платежа: {:.2f}'
    .format(orders_payed.query('is_recurrent == False')['total_price'].mean())
)

## Выводы

**Описание:**

Благотворительный фонд "АиФ Доброе сердце" хочет лучше узнать своих благотворителей, чтобы максимально эффективно выстроить свою работу с ними. 
У фонда есть «старожилы» - доноры, которые достаточно давно с фондом и одной из задач будет изучение их поведения: активны ли они, делают ли пожертвования и как реагируют на активность фонда?  

**Цели исследования:**

- проанализировать имеющиеся данные; 
- проанализировать поведение доноров: в целом, исторических пользователей и новых пользователей отдельно; 
- составить портрет типичного пользователя: регион, поведение, др. параметры; 
- рассчитать метрики: пользовательские, маркетинговые, коммерческие;
- сегментировать пользователей: RFM-анализ, описание сегментов;
- изучить каналы привлечения пользователей и их эффективность;
- изучить эффективность рассылок.

**Задачи:**

- провести исследовательский анализ данных;
- составить RFM - анализ пользователей;
- посчитать метрики пользователей;
- формулирование выводов по полученным результатам, подготовка презентации.

**Предобработка данных:** 

Данные предоставлены в хорошем качестве.

Для удобства работы:
- переменованы столбцы;
- подкорректирован тип данных, согласно заявленным значениям;
- добавлены новые столбцы с датами;
- обработаны пропуски, для корректной работы pandas;
- удалены лишние столбцы;
- удалены дубликаты;
- переформированы индексы.

**Пользовательские метрики:**

Уникальных пользователей: 23820, из которых 7343 – исторического импорта и 16477 – новых. Рекуренты составляют 2.7% от общего количества, волонтеры – менее 1%. Пол пользователей распределяется следующим образом: около 60% женщины, 20-30% мужчины (в зависимости от рекурентности), и у 10-20% пол не определен. Меньше чем у 10% пользователей подтверждена электронная почта. Тайм-зона у большинства пользователей не определена, либо указана как Europe/Moscow. Регион проживания также в большинстве случаев не определен, затем идет Vienna (возможно, из-за использования VPN) и Москва. Почти 99% пользователей подписаны через email. 75% пользователей исторического импорта проявляют активность, но только 8.6% из них являются платящими клиентами, а рекуренты среди них составляют 3%. Волонтеров среди исторического импорта нет. 

Первым касанием с пользователем чаще всего является первая оплата (или ее попытка), при которой фонд получает контактные данные пользователя для последующих рассылок. Для 10% пользователей есть активности без оплат, из которых треть – исторический импорт. Первая активность новых пользователей чаще всего происходит через сайт или email. Всплески первых активностей новых пользователей наблюдаются в августе 2022 года, феврале 2024 и сентябре 2023, совпадая с пиками первых оплат. 

Анализ графиков DAU (Daily Active Users) и MAU (Monthly Active Users) показывает тенденцию к росту, указывая на стабильное увеличение пользовательской базы и вовлеченности. Локальные падения активности наблюдаются в феврале 2022 и 2023 годов, вероятно, из-за сезонных факторов или внешних событий. Значительный рост в конце 2021 и середине 2022 года, вероятно, вызван успешными маркетинговыми кампаниями или улучшениями в сервисе. Максимальный пик активности отмечен в начале 2024 года, что может свидетельствовать о крупных обновлениях или эффективных стратегиях привлечения пользователей.

**Коммерческие метрики:**
Среднее пожертвование пользователя варьируется от 110 до 500 рублей, при среднем количестве оплат на одного пользователя от 1 до 3. Среднее количество платежей в день колеблется от 19 до 53. Самый высокий средний чек наблюдался в апреле 2024 года, а также в феврале и марте 2023 года, тогда как самый низкий средний чек был зафиксирован в феврале 2022, январе 2024 и ноябре 2021 года. Наибольший средний чек по назначению платежа был у Александра Слепухина, Дарьи Тортоевой и Никиты Яковлева. 

Главным источником входа пользователей чаще всего является сайт, где клиент делает пожертвование и оставляет свои контактные данные, что можно считать первым касанием с пользователем. Пользователи из "исторического импорта" совершили свое касание после импорта данных до мая 2022 года, после чего не было новых вхождений исторического импорта. Новые пользователи активно подключаются к фонду. 

Самыми активными днями недели по количеству пожертвований являются вторник и пятница, наименее активными – суббота, воскресенье и понедельник. Чаще всего пожертвования происходят в ночное время суток, реже всего утром. Самыми активными месяцами за всю историю по количеству пожертвований стали февраль 2024, декабрь 2023 и сентябрь 2023 года.


**Маркетинговые метрики:**
Первое касание пользователя чаще всего сопровождалось действием оплаты заказа. Первая активность клиента редко попадала в кампании, но если такое случалось, то самым популярным ресурсом для первого входа был VK, а самой популярной кампанией – Demid Lebedev. 

Каналы делятся на два вида: те, через которые проводятся кампании (utm_term Не указан, lina, vse, ida, google.com, first-stage, roditeli, report, razovie, Yandex.ru, congratulations), и те, через которые кампании не проводят. Одна и та же кампания может работать на разных каналах привлечения. 

В 2024 году кампании с наибольшей активностью были с канала utm_term Не указан, но по эффективности этот канал находится в середине списка. Самым эффективным каналом привлечения стал Ida, где почти каждая четвертая активность приводила к пожертвованию. Email – очень неэффективный канал привлечения, только 0.66% всех активностей заканчиваются пожертвованием.

**Рекомендации:**

Расширение базы точек контактов доноров может помочь вернуть "спящих клиентов". Необходимо расширять каналы привлечения пользователей. Основной канал связи с пользователями – Email, его эффективность составляет всего 0.66%, что указывает на низкую конверсию активностей в платежи. Наиболее эффективным каналом стал Ida, реклама у известных блогеров хорошо привлекает новых пользователей, однако без данных по затратам сложно оценить окупаемость полученных платежей.

Важно собирать номера мобильных телефонов при регистрации или активном действии, объясняя это необходимостью более оперативного и удобного информирования. Использование SMS-уведомлений и мессенджеров с напоминаниями о пожертвованиях, ссылками на отчеты и новостями может значительно повысить вовлеченность.

Стимулирование рекуррентных пожертвований играет ключевую роль. Нужно объяснять значимость регулярных пожертвований, обеспечивать простую настройку и отмену регулярных платежей. Возможно использование идей для стимулирования, таких как округление счетов совместно с банками. Программы лояльности и поощрения, прозрачность и отчетность, а также мобильные приложения и автоматизация помогут в этом процессе.

Необходимо оптимизировать частоту и контент сообщений, чтобы они были краткими и информативными, избегая чрезмерного количества отправок. Использование данных для создания персонализированных предложений и благодарностей, постоянный анализ поведения доноров и обновление RFM-анализа, а также прозрачная отчетность о том, как используются пожертвования, будут способствовать повышению доверия и вовлеченности доноров в деятельность фонда.

Эти шаги помогут эффективно предложить рекуррентные платежи донорам, которые готовы к этому, и обеспечат стабильное финансирование для реализации благотворительных проектов.






